### This file crops the fish eyes from the overall images and stores them in folders according to their class.

In [1]:
import os
from xml.etree import ElementTree as ET
from PIL import Image

def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find('filename').text
    objects = root.findall('.//object')

    annotations = []
    for obj in objects:
        class_name = obj.find('name').text
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        annotations.append({
            'class': class_name,
            'bbox': (xmin, ymin, xmax, ymax)
        })

    return filename, annotations

def crop_and_save_image(image_path, output_folder, filename, annotations):
    img = Image.open(image_path)
    for annotation in annotations:
        class_name = annotation['class']
        bbox = annotation['bbox']
        cropped_img = img.crop(bbox)

        class_folder = os.path.join(output_folder, class_name)
        os.makedirs(class_folder, exist_ok=True)

        save_path = os.path.join(class_folder, f"{filename}_{class_name}.jpg")
        cropped_img.save(save_path)

if __name__ == "__main__":
    input_folder = "./fish_class_3.v1i.voc/train/"  # Replace with the path to your folder
    output_folder = "./cropped_folder/"  # Replace with the path to your output folder

    for xml_file in os.listdir(input_folder):
        if xml_file.endswith(".xml"):
            xml_path = os.path.join(input_folder, xml_file)
            image_filename, annotations = parse_xml(xml_path)

            image_path = os.path.join(input_folder, image_filename)
            crop_and_save_image(image_path, output_folder, image_filename, annotations)
